# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [1]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-12-03 21:02:43--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   141MB/s    in 0.3s    

2024-12-03 21:02:44 (141 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [2]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [3]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [4]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [6]:
def map1(x):
  # Esta função deve retornar uma tupla (sentiment, id).
  # o argumento de entrada x é uma linha do dataframe imdb_df
  # com os dados em um formato semelhante à um dicionário
  # Apague a linha abaixo para iniciar seu código.
    # Retorna uma tupla onde o sentimento é a chave e o ID é o valor convertido
    # para inteiro
    return (x['sentiment'], int(x['id']))


## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [7]:
def reduceByKey1(x,y):
  # Coloque aqui o seu código para retornar o resultado necessário.
  # x e y são valores que serão somados, pois o reduceByKey receberá
  # apenas o segundo elemento da tupla vinda da saída da função map.
  # Apague a linha abaixo para iniciar seu código.
  return x + y

## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [8]:
# Linha de código para aplicar o map/reduce no seu dataframe spark
resultado = imdb_df.rdd.map(map1).reduceByKey(reduceByKey1).collect()
# Coloque aqui o código para imprimir o resultado. Não esqueça seu RU:
print("RU: 4670177")

print("resultado do processamento (sentiment, soma dos ids):")
for item in resultado:
  print(item)

RU: 4670177
resultado do processamento (sentiment, soma dos ids):
('neg', 459568555)
('pos', 763600041)


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [18]:
def map2(x):
  # Coloque aqui o seu código para retornar a tupla necessária.
  # Sugerimos o retorno em uma tupla com este formato:
  #   (sentiment, (quantidade_de_palavras_em_en, quantidade_de_palavras_em_pt))
  #   OU
  #   (sentiment, quantidade_de_caracteres_em_pt-quantidade_de_caracteres_em_en)
  # Apague a linha abaixo para iniciar seu código.
    # Contar palavras em inglês e português
    palavras_en = len(x['text_en'].split()) if x['text_en'] else 0
    palavras_pt = len(x['text_pt'].split()) if x['text_pt'] else 0

    # Retornar uma tupla (sentiment, (palavras_en, palavras_pt))
    return (x['sentiment'], (palavras_en, palavras_pt))




## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [16]:
def reduceByKey2(x,y):
  # Coloque aqui o seu código para retornar o resultado necessário.
  # x e y são valores que podem ser ou a tupla vinda da saída da função map
  # contendo quantidade de palavras em inglês e português, ou a diferença, a
  # depender da sua implementação da função map2.
  # Apague a linha abaixo para iniciar seu código.
    # Somar as palavras para inglês e português
    palavras_en_total = x[0] + y[0]
    palavras_pt_total = x[1] + y[1]
    return (palavras_en_total, palavras_pt_total)



## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [19]:
# Linha de código para aplicar o map/reduce no seu dataframe spark
resultado = imdb_df.rdd.map(map2).reduceByKey(reduceByKey2).collect()
# Coloque aqui suas linhas de código final para imprimir o resultado.
# Não esqueça seu RU:
# Filtrar apenas as linhas com 'sentiment' == 'neg'
rdd_filtrado = imdb_df.rdd.filter(lambda x: x['sentiment'] == 'neg')

# Aplicar as funções map e reduceByKey
resultado = rdd_filtrado.map(map2).reduceByKey(reduceByKey2).collect()

# Processar o resultado final
for sentiment, (total_en, total_pt) in resultado:
    if sentiment == 'neg':
        diferenca = total_pt - total_en
        print(f"Resultado final (RU: 4670177):")
        print(f"Total palavras em português: {total_pt}")
        print(f"Total palavras em inglês: {total_en}")
        print(f"Diferença (pt - en): {diferenca}")



Resultado final (RU: 4670177):
Total palavras em português: 5455273
Total palavras em inglês: 5400324
Diferença (pt - en): 54949
